# How to...use DbEngine to execute a simple query

In [1]:
# Sets up the location of the api relative to this notebook 
import sys
sys.path.append('../../../')

In [4]:
# Import the module for connection to a sqllite database
from esg_matching.engine.connectors.sql_lite import SqlLiteConnector

In [5]:
# Import the module for datasource reflection
from esg_matching.data_source.db_source import DbDataSource

## 1. Database setup

In [6]:
# Localization of the database to be created in relation to this jupyter notebook
# The database will be created in the /data/dabase folder, under the project main folder
path_db = '../../../tests/data/howto/sqlite/test_esg_matching.db'

In [7]:
# The database connector is represented by the class SqlLiteConnector 
db_conn = SqlLiteConnector()

In [8]:
# The connect() method of the SqlLiteConnector is used to stablish a connection with the database if it exists, 
# or to create a new one. The property path_db defines the location and name of the database.
# The  property show_sql_statement indicates if the SQL statements are echoed (or printed) in the default output channel.
db_conn.path_db = path_db
db_conn.show_sql_statement = False
db_conn.connect()

In [9]:
# Check if the connection was stablished
db_conn.is_connected()

True

## 2. Using DbDataSource to reflect an existent table 

In [10]:
# Create a datasource object
my_table = DbDataSource(db_conn, 'ds_matching')

In [11]:
# Set up important properties before syncronizing the database table
my_table.table_name = 'esg_matching'

In [12]:
# Sync with database table and bring the attribute names and pks
my_table.sync_with_db_table(set_original_fields=True, set_pks=True)

In [13]:
# Checking the attributes
my_table.get_attribute_names()

[Column('timestamp', DATETIME(), table=<esg_matching>, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x000001E14EFA3580>, for_update=False)),
 Column('matching_id', INTEGER(), table=<esg_matching>, primary_key=True, nullable=False),
 Column('ref_name', VARCHAR(), table=<esg_matching>),
 Column('tgt_name', VARCHAR(), table=<esg_matching>),
 Column('matching_type', VARCHAR(), table=<esg_matching>),
 Column('matching_scope', VARCHAR(), table=<esg_matching>),
 Column('matching_rule', VARCHAR(), table=<esg_matching>),
 Column('ref_id', INTEGER(), table=<esg_matching>),
 Column('ref_company', VARCHAR(), table=<esg_matching>),
 Column('ref_country', VARCHAR(), table=<esg_matching>),
 Column('tgt_id', INTEGER(), table=<esg_matching>),
 Column('tgt_company', VARCHAR(), table=<esg_matching>),
 Column('tgt_country', VARCHAR(), table=<esg_matching>),
 Column('isin', VARCHAR(), table=<esg_matching>),
 Column('lei', VARCHAR(), table=<esg_matching>),
 Column('sedol', VARC

## 3. Build a query

In [14]:
# Import the query builder module 
from esg_matching.engine.builders.sql_builder import SelectBuilder, SqlConditionBuilder
from esg_matching.engine.sql.dql import DqlManager

In [15]:
# Instantiate the condition builder as to construct where clauses for your select statement
condition_builder = SqlConditionBuilder()

In [16]:
# Get the reference to the column in the datasource to be used in the where clause
col_tgt = my_table.get_table_column('tgt_name')
col_tgt

Column('tgt_name', VARCHAR(), table=<esg_matching>)

In [17]:
# Build the where condition clause
where_condition = condition_builder.create_condition().equal_value(col_tgt, 'DS-TGT1').get_condition()
print(str(where_condition))

esg_matching.tgt_name = :tgt_name_1


In [18]:
# Instantiate the select builder as to construct select statements
sql_builder = SelectBuilder()

In [19]:
# Define the names of the column for your select and get the table reference to them 
col_names = ['ref_id', 'ref_company', 'ref_country', 'tgt_id', 'tgt_company', 'tgt_country', 'isin', 'lei', 'sedol']
col_alias = ['ref_id', 'ref_entity', 'ref_country', 'tgt_id', 'tgt_entity', 'tgt_country', 'isin_id', 'lei_id', 'sedol_id']
cols_db = my_table.get_table_columns(col_names, col_alias)

In [ ]:
# Uncomment the next line to select the columns without an alias
# cols_db = my_table.get_table_columns(col_names)

In [20]:
# Build the sql statement
sql_stm = sql_builder.create_select(cols_db).from_table(my_table.table_obj).where_condition(where_condition).build_statement()
print(str(sql_stm))

SELECT esg_matching.ref_id AS ref_id, esg_matching.ref_company AS ref_entity, esg_matching.ref_country AS ref_country, esg_matching.tgt_id AS tgt_id, esg_matching.tgt_company AS tgt_entity, esg_matching.tgt_country AS tgt_country, esg_matching.isin AS isin_id, esg_matching.lei AS lei_id, esg_matching.sedol AS sedol_id 
FROM esg_matching 
WHERE esg_matching.tgt_name = ?


In [21]:
# Create the Data Query Language (DQL) manager object that allows to execute queries in the database
sql_executor = DqlManager(db_conn)

In [22]:
# Execute the query and receive data as a list of tuples
result_data = sql_executor.query_by_sql_statement(sql_stm)
result_data

[]

In [23]:
# Execute the query and receive data as a pandas dataframe
result_data = sql_executor.query_by_sql_statement(sql_stm, as_pandas_df=True)
result_data

,ref_id,ref_entity,ref_country,tgt_id,tgt_entity,tgt_country,isin_id,lei_id,sedol_id
